In [ ]:
#Proceso05
#visualizar mapa palermo, deptos

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import geopandas
import fiona
import rtree
import folium

from shapely.geometry import Point

pd.options.display.max_rows = 10

In [ ]:
def from_wkt(df, wkt_column):
    import shapely.wkt
    df["coordinates"]= df[wkt_column].apply(shapely.wkt.loads)
    gdf = geopandas.GeoDataFrame(barrios, geometry='coordinates')
    return gdf

def from_x_y(df, x, y):
    gdf = geopandas.GeoDataFrame(df.drop([x, y], axis=1),
                                crs={'init': 'epsg:4326'},
                                geometry=[Point(xy) for xy in zip(df[x], df[y])])
    return gdf

In [ ]:
barrios = pd.read_csv("../data/barrios.csv", encoding='latin1')

barrios = from_wkt(barrios, "WKT")

In [ ]:
barrios.head()

In [ ]:
propiedades = pd.read_csv('../data/out_proceso02.csv', decimal=".")

propiedades = propiedades[(propiedades['provincia'] == 'capital federal') & 
                          (propiedades['tipo_propiedad'] == 'departamento') & 
                          (propiedades['emprendimiento'] == False) & 
                          (propiedades['en_pozo'] == False) &
                          (propiedades['ciudad_barrio'] == 'palermo') &
                          (propiedades['lat'].notnull()) &
                          (propiedades['lon'].notnull()) &
                          (propiedades['sup_m2'] >= 50) &
                          (propiedades['sup_m2'] <= 80)                          
                         ]

propiedades = from_x_y(propiedades, 'lon', 'lat')

In [ ]:
propiedades.groupby('tipo_propiedad').count()

In [ ]:
m = folium.Map([-34.606359, -58.443863], zoom_start=12, tiles="OpenStreetMap")
folium.GeoJson(barrios.to_json()).add_to(m)
folium.GeoJson(propiedades.to_json()).add_to(m)
m